# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_routing import *

In [ ]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Figures

## Domain and altitude

In [ ]:
import psyplot.project as psy
psy.rcParams['auto_show'] = True
mpl.rcParams['figure.figsize'] = [8, 6.]

In [ ]:
LAM_dir='../../../JZ_simu_outputs/LAM/native_outputs/'
start_file='{}/espagne_start_2010.nc'.format(LAM_dir)
start=xr.open_dataset(start_file)
# start

In [ ]:
start["Altitude"] = start["phis"]/9.81
start["Altitude"].attrs["units"] = "m"
start["Altitude"].attrs["long_name"] = "Altitude"
start["Altitude"].attrs["standard_name"] = "Altitude"

start_file2='{}/espagne_start_2010_modified.nc'.format(LAM_dir)
start.to_netcdf(start_file2)

In [ ]:
start2=xr.open_dataset(start_file2)
# start2

In [ ]:
# Custom colormap
terrain_colors = plt.cm.terrain(np.linspace(0, 1, 25))
filtered_colors = terrain_colors[5:]  
# Insert blue as the first color
filtered_colors = np.vstack([[0.07973856209150328, 0.44052287581699345, 0.8405228758169935, 1.0], filtered_colors]) 
custom_colormap = ListedColormap(filtered_colors)


In [ ]:
savefig=True
file=start_file2
var='Altitude'
vmin=-0
vmax=2000
pas=(vmax-vmin) / 20

map=psy.plot.mapplot(
    file, 
    name=var,
    datagrid=dict(color='k', linewidth=0.2),
    cbar='r',
    tight=True,
    lsm='50m',
    cmap=custom_colormap,
    extend='both',
    projection='ortho',
    xgrid=True,ygrid=True,
    bounds=np.arange(vmin, vmax + pas, pas),
    # title=var,
    clabel="Altitude (m)"
)

if savefig:
    plt.savefig('figures/methods/altitude_map.png', dpi=300, bbox_inches='tight')
    print("Figure saved as 'figures/altitude_map.png'")

## Irrigated fraction and AEI_SW

In [ ]:
savefig=True

In [ ]:
irr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/irr'
filename='{}/*/SRF/MO/*2013*history.nc'.format(irr_dir)

ORCirr0 = xr.open_mfdataset(filename)
ORCirr = ORCirr0.rename({'time_counter':'time'})
ORCirr.attrs['name'] = 'irr'
ORCirr.attrs['plot_color'] = "#0C5DA5"
ORCirr = ORCirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

ORCirr['irrig_frac'] = ORCirr['irrigmap_dyn'] / ORCirr['Areas'] * 100
ORCirr['irrig_frac'].attrs['units'] = '%'
ORCirr['irrig_frac'].attrs['long_name'] = 'Irrigated fraction'

ORCirr['aei_sw'] = ORCirr['aei_sw'] * 100

ORCirr

In [ ]:
filename = '{}/*/ATM/MO/*2013*.nc'.format(irr_dir)


lmdz = xr.open_mfdataset(filename)
lmdz = lmdz.rename({'time_counter':'time'})
lmdz = lmdz.sel(lon=slice(lon_min, lon_max),lat=slice(lat_min, lat_max))

lmdz

In [ ]:
ip_mask = polygon_to_mask(ORCirr, iberian_peninsula)
con_mask=lmdz.mean(dim='time')['contfracOR']>0.95

ip_ORCirr = ORCirr.where(ip_mask, drop=True)
contIP_ORCirr = ip_ORCirr.where(con_mask)

In [ ]:
var='irrig_frac'
vmin=0
vmax=50
# vmin, vmax = None, None  # Use default min/max for the colormap
ds=contIP_ORCirr
map_ave(ds, var, vmin, vmax, cmap=bluesW,
        title='off', clabel='Grid cell irrigated fraction (%)')
if savefig:
        plt.savefig('figures/methods/irrigated_fraction_map.png', dpi=300, bbox_inches='tight')
        print("Figure saved")

In [ ]:
var='aei_sw'
vmin=0
vmax=100
# vmin, vmax = None, None  # Use default min/max for the colormap
ds=contIP_ORCirr
map_ave(ds, var, vmin, vmax, cmap=emb_neutral,
        title='off', clabel='Share of surface equipments (%)')
if savefig:
        plt.savefig('figures/methods/aei_sw_map.png', dpi=300, bbox_inches='tight')
        print("Figure saved")

In [ ]:
var='aei_sw'
vmin=0
vmax=100
# vmin, vmax = None, None  # Use default min/max for the colormap
ds=contIP_ORCirr
map_ave(ds, var, vmin, vmax, cmap=emb_neutral,
        title='off', clabel='Share of surface equipments (%)')

## Routing DEM

In [ ]:
filename='../../../input_data/routing.nc'
std_halfdeg = xr.open_dataset(filename)
#remap y axis, from (0,359) it should be (90N,90S)
std_halfdeg = std_halfdeg.rename({'y': 'lat'})
std_halfdeg = std_halfdeg.assign_coords(lat=89.75 - std_halfdeg.lat/2) 

#remap x axis, from (0,719) it should be (-180,180)
std_halfdeg = std_halfdeg.rename({'x': 'lon'})
std_halfdeg = std_halfdeg.assign_coords(lon=std_halfdeg.lon/2 -179.75)  

#select Iberian Peninsula zone
std_halfdeg = std_halfdeg.sel(lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min))

std_halfdeg

In [ ]:
filename='../../../input_data/routing_simple.nc'
interp_halfdeg = xr.open_dataset(filename)

#select Iberian Peninsula zone
interp_halfdeg = interp_halfdeg.sel(lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min))

interp_halfdeg


In [ ]:
vmin, vmax=1,10
map_plotvar(std_halfdeg['trip'], vmin=vmin, vmax=vmax, title='routing.nc trip')

In [ ]:
vmin, vmax=0,9000
# vmin, vmax=1,8
# vmin, vmax=None, None  # Use default min/max for the colormap
cmap=redsW
var='topoind'
map_plotvar(interp_halfdeg[var], vmin=vmin, vmax=vmax, title=f'routing_simple.nc {var}',
            cmap=cmap)

In [ ]:
var1='trip'
var2='trip_extended'
# var2=var1
vmin=-1
vmax= 1
# vmin, vmax = None, None  # Use default min/max for the colormap
diff= interp_halfdeg['trip_extended'] - std_halfdeg[var1]
map_plotvar(diff, cmap=emb_neutral,
            vmin=vmin, vmax=vmax,
            title='Difference between trip (routing.nc) and trip_extended (routing_simple.nc) ')

In [ ]:
filename='../../../input_data/routing_ext_merit.nc'
ext_halfdeg = xr.open_dataset(filename)
#select Iberian Peninsula zone
ext_halfdeg = ext_halfdeg.sel(lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min))
ext_halfdeg

In [ ]:
vmin=0
vmax=9000
# vmin, vmax = None, None  # Use default min/max for the colormap
map_plotvar(ext_halfdeg['topoind'], vmin=vmin, vmax=vmax, title='routing_ext_merit.nc topoind')